In [9]:
# Install libraries for data_loader
!pip install pandas
!pip install sklearn
#!pip install albumentations
!pip install torchvision
!pip install tqdm
!pip install torchmetrics

In [4]:
# Import DataLoader and corresponding libraries
import pandas
import torchvision.transforms as TT
# import albumentations as T
# import albumentations.augmentations.transforms as T_transforms
# from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, Dataset, sampler
from sklearn.model_selection import StratifiedKFold
from torchvision import utils
from PIL import Image

In [8]:
# Import libraries for tensors
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# For fbeta-score
from torchmetrics.functional import fbeta_score

# For nn.Sequential
from collections import OrderedDict

In [9]:
# Import tqdm for progress bar construction
import tqdm

In [10]:
# Datatypes and Devices (from Assignment 2)
dtype = torch.float
ltype = torch.long

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cpu


In [29]:
# Hyper-parameters for K-Fold Cross Validation
N = 5
seed = 42

# Directories for Data
FF1010_Path = 'C:\\users\\ujubo\\birdnocall\\'
AudioImage_Path = './image/'

In [30]:
# Call StratifiedKFold object
skf = StratifiedKFold(
    n_splits=5, shuffle=True, random_state=seed
)

In [34]:
# Modify dataframe for K-Fold Cross Validation (ff1010)
ff1010_csv = pandas.read_csv(FF1010_Path + 'metadata.csv')
#no call
ff1010_csv.loc[ff1010_csv['hasbird'] == 0, 'filepath'] = \
    FF1010_Path + 'nocall/' + ff1010_csv.query('hasbird == 0')['filename'] + '.npy'
#call
ff1010_csv.loc[ff1010_csv['hasbird'] == 1, 'filepath'] = \
    FF1010_Path + 'bird/' + ff1010_csv.query('hasbird == 1')['filename'] + '.npy'


ff1010_csv = ff1010_csv.dropna()
ff1010_csv = ff1010_csv.reset_index(drop=True)

# Add 'fold' attribute for dataset classification
ff1010_dataframe = ff1010_csv.copy()
for n, (_, nth_groups) in enumerate(
    skf.split(ff1010_dataframe, ff1010_dataframe['hasbird'])):
    ff1010_dataframe.loc[nth_groups, 'fold'] = int(n)

12       C:\users\ujubo\birdnocall\bird/19037.wav.npy
32       C:\users\ujubo\birdnocall\bird/71838.wav.npy
34       C:\users\ujubo\birdnocall\bird/97375.wav.npy
38       C:\users\ujubo\birdnocall\bird/72827.wav.npy
42      C:\users\ujubo\birdnocall\bird/166175.wav.npy
                            ...                      
7668     C:\users\ujubo\birdnocall\bird/56522.wav.npy
7677     C:\users\ujubo\birdnocall\bird/70892.wav.npy
7679     C:\users\ujubo\birdnocall\bird/73281.wav.npy
7687     C:\users\ujubo\birdnocall\bird/80789.wav.npy
7688    C:\users\ujubo\birdnocall\bird/104733.wav.npy
Name: filepath, Length: 1935, dtype: object


In [32]:
# # Modify dataframe for K-Fold Cross Validation (birdclef2021)
# birdclef_csv = pandas.read_csv(AudioImage_Path + 'metadata.csv')
# birdclef_csv.loc[birdclef_csv['label_id'] >= 0,'filepath'] = \
#     AudioImage_Path + birdclef_csv.query('label_id >= 0')['primary_label'] + '/' + \
#     birdclef_csv.query('label_id >= 0')['filename'] + '.npy'

# birdclef_csv = birdclef_csv.dropna()
# birdclef_csv = birdclef_csv.reset_index(drop=True)

# # Add 'fold' attribute for dataset classification
# birdclef_dataframe = birdclef_csv.copy()
# for n, (_, nth_groups) in enumerate(
#     skf.split(birdclef_dataframe, birdclef_dataframe['label_id'])):
#     birdclef_dataframe.loc[nth_groups, 'fold'] = int(n)

In [33]:
# Hyper-parameters for training 
ff1010_batch = 32
birdclef_batch = 32

In [40]:
# Class for ff1010 dataset
class FF1010(Dataset):
    def __init__(self, dataframe, process='train', labels='hasbird'):
        self.dataframe = dataframe
        self.filepaths = dataframe['filepath'].values
        self.labels = dataframe[labels].values
        self.process = process
        
        # Transforms for each train and validation
        self.train_transform = TT.Compose([
            TT.Resize([128, 281]),
            TT.RandomHorizontalFlip(p=0.5),
            TT.RandomVerticalFlip(p=0.5),
#             T_transforms.ImageCompression(p=0.5, 
#                 compression_type=T_transforms.ImageCompression.ImageCompressionType.JPEG),
#             T_transforms.ImageCompression(p=0.5, 
#                 compression_type=T_transforms.ImageCompression.ImageCompressionType.WEBP),
            TT.ToTensor(),
            TT.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
        ])
        self.val_transform = TT.Compose([
            TT.Resize([128, 281]),
            TT.ToTensor(),
            TT.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
        ])
    
    def __getitem__(self, idx):
        source = np.load(self.filepaths[idx])
        
        # Rearrange numpy arrays
        source = source.transpose(1, 2, 0)
        # Add RGB dimension
        source = np.stack((np.squeeze(source), ) * 3, -1)
        
        # Apply transform
        if self.process == 'train':
            source = self.train_transform(Image.fromarray(source))
        elif self.process == 'valid':
            ource = self.val_transform(Image.fromarray(source))
        
        return source, torch.tensor(self.labels[idx], dtype=ltype).to(device)
    
    def __len__(self):
        return len(self.dataframe)

In [41]:
from time import sleep

# Train loop for nocall detector
def nocall_train(train_dataframe, val_dataframe):  
    train_data = FF1010(train_dataframe, process='train', labels='hasbird')
    val_data = FF1010(val_dataframe, process='valid', labels='hasbird')
    
    # Construct data loader for train and validation
    train_loader = DataLoader(train_data, batch_size=ff1010_batch,
                             sampler=sampler.SubsetRandomSampler(range(len(train_dataframe))), 
                             drop_last=True)
    val_loader = DataLoader(val_data, batch_size=ff1010_batch,
                             sampler=sampler.SubsetRandomSampler(range(len(val_dataframe))),
                             drop_last=False)
    
    # Test for loaders
    
    optimizer = optim.SGD(model_1.parameters(), lr=learning_rate_1, momentum=0.5, weight_decay=weight_decay_1)
    val_losses, train_losses = train_model(model_1, train_loader, val_loader, optimizer, beta=beta)
    
    return val_losses, train_losses

In [48]:
# Print period for accuracy.
print_period = 12

# Learning Rate.
learning_rate_1 = 0.001

# L2 Regularization Hyperparamter
weight_decay_1 = 0.001

# Beta constant for Fbeta-score.
# If you want to give more weight to precision, use value smaller than 1.0.
# If you want to give more weight to recall, use value larger than 1.0.
beta = 1.0

# Prototype of model 1.
# ResNet50 outputs (Batchsize, 1000) tensor as output, so we reduce them to 2.
# TODO: I'm curious about the output of the model: would output be the float number
#       between 0 and 1? If not, we need to add sigmoid or softmax function at the end.
model_1 = nn.Sequential(OrderedDict([
        ("resnet50", torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True).to(device)),
        ("relu", nn.ReLU().to(device)),
        ("linear", nn.Linear(1000, 2, bias=True).to(device))
]))

print(model_1)

def train_model(model, train_loader, val_loader, optimizer, beta=1.0, epoch=10):
    loss = 0
    log_score = 0
    for e in range(epoch):
        print(f"Training model 1, epoch {e+1}")
        for index, (source, label) in enumerate(tqdm.tqdm(train_loader)):
            x = source.to(device=device, dtype=torch.float)  # move to device, e.g. GPU
            y = label.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.nll_loss(F.log_softmax(scores), y) # Log loss for our project.

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # every print_period, evaluate on val_loader.
            if index % print_period == 0:
                print('Iteration %d, loss = %.4f' % (index, loss.item()))
                acc, log_score, fbeta_score = evaluate_model(val_loader, model, beta=beta)
                print()

    return log_score, loss

Using cache found in C:\Users\ujubo/.cache\torch\hub\pytorch_vision_v0.10.0


Sequential(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [45]:
# Train, Validate and Test for nocall detector
def nocall(dataframe, val_index, test_index):
    
    # Check that validation fold is not same as test fold
    assert val_index != test_index, \
        'Validation and test should be done on different fold.'
    
    train_dataframe = dataframe.query(
        'fold != ' + str(val_index) + ' and fold != ' + str(test_index) 
    ).reset_index(drop=True)
    val_dataframe = dataframe.query(
        'fold == ' + str(val_index) 
    ).reset_index(drop=False)
    
    val_losses, train_losses = nocall_train(train_dataframe, val_dataframe)
    
    # TODO
    # 이쪽에 Accuracy test 구현하시면 됩니다.
    
    return

In [49]:
nocall(ff1010_dataframe, 0, 1)

Training model 1, epoch 1


  0%|                                                                                          | 0/144 [00:00<?, ?it/s]C:\Users\ujubo\AppData\Local\Temp/ipykernel_19588/1450643.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = F.nll_loss(F.log_softmax(scores), y) # Log loss for our project.
  1%|▌                                                                                 | 1/144 [00:03<09:11,  3.86s/it]

Iteration 0, loss = 0.7806



  9%|███████▎                                                                         | 13/144 [00:50<08:34,  3.93s/it]

Iteration 12, loss = 0.4303



...

 84%|███████████████████████████████████████████████████████████████████▏            | 121/144 [07:46<01:28,  3.83s/it]

Iteration 120, loss = 0.2528



 92%|█████████████████████████████████████████████████████████████████████████▉      | 133/144 [08:32<00:42,  3.83s/it]

Iteration 132, loss = 0.2276



100%|████████████████████████████████████████████████████████████████████████████████| 144/144 [09:14<00:00,  3.85s/it]
